In [ ]:
%pylab qt

In [ ]:
from region.image import Image
from utils import *
import numpy
from skimage.draw import polygon_perimeter, polygon
from scipy.interpolate import interp2d, dfitpack

In [ ]:
np.set_printoptions(suppress=True)

# Utils

In [ ]:
def getPoints(im, n):
    n = int(n)
    plt.imshow(im.imArr)
    points = plt.ginput(n, timeout = 300, show_clicks=True)
    plt.close()
    print("Points gotten from", im.name, ":", str(points))
    return np.array(points)

def dispPtsOnImg(im, results, tag=False):
    dispPtsOnImgRC(im, results[:, 0], results[:, 1], tag)

def dispPtsOnImgRC(im, rowInds, colInds, tag=False):
    assert len(rowInds) == len(colInds)
    plt.imshow(im)
    plt.plot(rowInds, colInds, 'bo')
    if tag:
        for ind in range(len(rowInds)):
            x, y = rowInds[ind], colInds[ind]
            plt.text(x, y, str(ind), bbox=dict(facecolor='blue', alpha=0.5))
            plt.text(x, y, "%d-%d" % (x,y), bbox=dict(facecolor='red', alpha=0.5))
    plt.waitforbuttonpress()
    plt.close()

In [ ]:
imT1 = Image.fromPath("../sample_imgs/20181103_162858.jpg")

In [ ]:
viewImage(imT1.imArr)

In [ ]:
def computeH(startPts, endPts):
    assert len(startPts) == len(endPts)
    A, b = createA(startPts, endPts)
#     Axx = createAb(startPts, endPts)
#     bxx = np.zeros(len(Axx))
#     bxx[-1] = 1
#     print("whoomp: ", numpy.linalg.lstsq(Axx, bxx)[0])
    
    x = numpy.linalg.lstsq(A, b)
    result = np.append(x[0], 1)
    return result.reshape(3, 3)

In [ ]:
computeH(im1Pts, im2Pts)

In [ ]:
imSproul0 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg", 0.25)
imSproul1 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_01.jpg", 0.25)

In [ ]:
def createAb(startPts, endPts):
    result = np.zeros((len(startPts) * 2 + 1, 9))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        result[i*2, 8] = endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        result[i*2+1, 8] = endPoint[1]
    result[-1, -1] = 1
    
    return result


def createA(startPts, endPts):
    result = np.zeros((len(startPts) * 2, 8))
    b = np.zeros(len(result))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        b[i*2] = -endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        b[i*2+1] = -endPoint[1]
    # result[-1, -1] = 1
    
    return result, b
    

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

In [ ]:
dispPtsOnImg(imSproul0.imArr, im1Pts)
dispPtsOnImg(imSproul1.imArr, im2Pts)

In [ ]:
im1Pts = 1/4 * np.array([[515, 1477], [514, 1732], [333, 1668], [691, 1619], [791, 1605], [1286, 1535], [2038, 1408], [2032, 1528]])
im2Pts = 1/4 * np.array([[1911, 1379], [1908, 1615], [1712, 1542], [2056, 1519], [2140, 1515], [2598, 1488], [3430, 1416], [3421, 1545]])
# im1Pts = np.array([])
# im2Pts = np.array([])
# createA(im1Pts, im2Pts)

In [ ]:
imSource = imSproul0


"""
p' = H p
where
    p' = target
    p = source

place on p' the color on p

---

get the points in final image that actually will need a value from source image
    do by calculating the results of the corners of the source img getting transformed,
    then using polygon to get the coordinates that actually lies inside the polygon, so you dont do the coords that isnt in it
    

for every point in p', get the corresponding p with p' * H^-1

"""

In [ ]:
sourcePts = im1Pts
targetPts = im2Pts

H = computeH(sourcePts, targetPts)
print("source:", sourcePts)
print("transformed source:", transf(H, sourcePts))
print("target:", targetPts)

dispPtsOnImg(imSproul0.imArr, sourcePts)
dispPtsOnImg(imSproul1.imArr, transf(H, sourcePts))

In [ ]:
def transf(H, res, normalize=True):
    if res.ndim == 1:
        vect = np.append(res, 1)
    else:
        vect = np.vstack((res.T, np.ones(len(res))))
    tfSourceCorners = np.dot(H, vect)
    
    if normalize:
        for i in range(tfSourceCorners.shape[1]):
            corner = tfSourceCorners[:, i]
            normCorner = np.dot(corner, 1/(corner[2]))
            tfSourceCorners[:, i] = normCorner
    return tfSourceCorners[:-1, :].T

In [ ]:
def getDimensions(imCornersCoords):
    # assuming imCornersCoords = [[width, height]]
    
    minWidth = int(np.floor(min(imCornersCoords[:, 0])))
    minHeight = int(np.floor(min(imCornersCoords[:, 1])))
    
    maxWidth = int(np.ceil(max(imCornersCoords[:, 0])))
    maxHeight = int(np.ceil(max(imCornersCoords[:, 1])))
    
    totalWidth = maxWidth - minWidth + 1
    totalHeight = maxHeight - minHeight + 1
    
    tup = totalHeight, totalWidth, minHeight, minWidth
    print(tup)
    
    return tup
    

In [ ]:
def findAllCoords(tfSourceCorners):
    innerCoordsR, innerCoordsC = polygon(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
    perimCoordsR, perimCoordsC = polygon_perimeter(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
    allCoordsR = np.append(innerCoordsR, perimCoordsR)
    allCoordsC = np.append(innerCoordsC, perimCoordsC)

    targetPs = np.vstack((allCoordsR, allCoordsC)).T
    
    return targetPs

In [ ]:
class WarpedImageInfo:
    
    def __init__(self, corners, targetPs, rvals, gvals, bvals):
        self.corners = corners
        self.targetPs = targetPs
        self.rvals = rvals
        self.gvals = gvals
        self.bvals = bvals

def warpTo(imSource, sourcePts, targetPts):

    H = computeH(sourcePts, targetPts)
    sourceCorners = Image.getCorners(imSource)
    tfSourceCorners = transf(H, sourceCorners)

#     print("sourceCorners:", sourceCorners)
#     print("tfSourceCorners:", tfSourceCorners)

    targetPs = findAllCoords(tfSourceCorners)

    invH = np.linalg.inv(H)

    sourcePs = transf(invH, targetPs)
    
    source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
    source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
    source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])
    autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

    Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
    Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
    Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
    
    
    return WarpedImageInfo(tfSourceCorners, targetPs, Rvals, Gvals, Bvals)

# finalImage = warpTo(imSproul0, im1Pts, imSproul1, im2Pts)
# finalImage2 = warpTo(imSproul0, im1Pts, imSproul0, im1Pts)

# args
imgList = [(imSproul0, im1Pts), (imSproul1, im2Pts)]
imBase = imSproul1
imBasePts = im2Pts

warpedResults = []
for i in range(len(imgList)):
    
    im, imPts = imgList[i]
    
    warpedResults.append(warpTo(im, imPts, imBasePts))
    
allCorners = np.vstack([warped.corners for warped in warpedResults])
print(allCorners)
finH, finW, minH, minW = getDimensions(allCorners)
finalImage = Image("finalImage", np.zeros((finH, finW, 3)))

newWarpedImages = []
# for warped in warpedResults:
for i in range(len(warpedResults)):
    warped = warpedResults[i]
    newImage = Image("finalImage", np.zeros((finH, finW, 3)))
    newImage.imArr[warped.targetPs[:, 1] + minH, warped.targetPs[:, 0] + minW, 0] = warped.rvals
    newImage.imArr[warped.targetPs[:, 1] + minH, warped.targetPs[:, 0] + minW, 1] = warped.gvals
    newImage.imArr[warped.targetPs[:, 1] + minH, warped.targetPs[:, 0] + minW, 2] = warped.bvals
    printImage(str(i) + ".png", newImage.imArr)

In [ ]:
imTestImage = Image.fromPath("../curated_sample_imgs/test_image.png")
imSource = imTestImage

source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])

autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

######

sourceCorners = np.array([[0, 0], [0, 479], [479, 479], [479, 0]])

tfSourceCorners = np.array([[99, 99], [99, 579], [579, 579], [579, 99]])

H = computeH(sourceCorners, tfSourceCorners)
print(H)


###


innerCoordsR, innerCoordsC = polygon(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
perimCoordsR, perimCoordsC = polygon_perimeter(tfSourceCorners[:, 0], tfSourceCorners[:, 1])
allCoordsR = np.append(innerCoordsR, perimCoordsR)
allCoordsC = np.append(innerCoordsC, perimCoordsC)

targetPs = np.vstack((allCoordsC, allCoordsR)).T


invH = np.linalg.inv(H)

sourcePs = transf(invH, targetPs)


finH = int(np.ceil(max(targetPs[:, 1])))
finW = int(np.ceil(max(targetPs[:, 0])))
finalImage = Image("finalImage", np.zeros((finH + 1, finW + 1, 3)))
print(finalImage.imArr.shape)

Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
print(targetPs)

finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 0] = Rvals
finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 1] = Gvals
finalImage.imArr[targetPs[:, 1], targetPs[:, 0], 2] = Bvals

viewImage(finalImage.imArr)

In [ ]:
from skimage.draw import polygon_perimeter
img = np.zeros((10, 10), dtype=np.uint8)
rr, cc = polygon_perimeter([6, 1, 5, 10],
                           [1, 5, 11, 5],
                           shape=img.shape, clip=True)
img[rr, cc] = 1
img

In [ ]:
np.append(np.array([1, 2, 3]), 1)

In [ ]:
a = np.array([[0, 0]])
b = np.array([[2, 2]])
print(createA(a, b))
print(createAb(a, b))